In [1]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit=false"
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1 inter_op_parallelism_threads=1"

In [2]:
from tensorflow import keras
import tensorflow as tf
from datetime import datetime

2025-03-15 20:11:47.650159: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-15 20:11:47.660255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742069507.672059    2119 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742069507.675409    2119 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-15 20:11:47.688991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from Pills.GenerativeSequence import GenerativeSequence
from PillNet import PillNet

In [4]:
SOURCE_PATH = '/workspaces/dev/datasets/pills/data'
LABEL_PATH = '/workspaces/dev/datasets/pills/class_label.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data'

In [5]:
TRAIN_DATA_SIZE = 32
VALIDATION_DATA_SIZE = 4
BATCH_SIZE = 4
INPUT_SIZE = (256, 256, 3)

In [6]:
model = PillNet(50)
model.compile(jit_compile=False)
# model.compile(jit_compile=False, run_eagerly=True)
model.build(BATCH_SIZE, INPUT_SIZE)
model.summary()

I0000 00:00:1742069510.147302    2119 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13512 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "pill_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ROI (Sequential)                │ (None, 128, 128, 1)    │        98,705 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DetectionHead (Sequential)      │ (None, 8, 8, 4)        │       132,860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FeatureExtractor_1 (Sequential) │ (None, 128, 128, 32)   │         9,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FeatureExtractor_2 (Sequential) │ (None, 128, 128, 32)   │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FeatureExtractor_3 (Sequential) │ (None, 128, 128, 32)   │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CentroidHead (Sequential)       │ (None, 2)              │         8,126 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SegmentationHead (Sequential)   │ (None, 128, 128, 1)    │        29,777 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ClassificationHead (Sequential) │ (None, 50)             │       159,528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,525,886 (5.82 MB)

 Trainable params: 508,628 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,017,258 (3.88 MB)

In [7]:
train_sequence = GenerativeSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)
validation_sequence = GenerativeSequence(
  VALIDATION_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2]), dtype=tf.float32),
    {
      "roi": tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1]), dtype=tf.bool),
      "centroid": tf.TensorSpec(shape=(BATCH_SIZE * 16, 2), dtype=tf.float32),
      "detection": tf.TensorSpec(shape=(BATCH_SIZE * 16, 4), dtype=tf.float32),
      "segmentation": tf.TensorSpec(shape=(BATCH_SIZE * 16, INPUT_SIZE[0]//4, INPUT_SIZE[1]//4), dtype=tf.bool),
      "classification": tf.TensorSpec(shape=(BATCH_SIZE * 16), dtype=tf.uint32)
    }
  )
)

In [8]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath=f"/workspaces/dev/models/R-CNN/checkpoints/pill_net_{datetime.now().strftime('%Y_%m_%d')}.keras",
  monitor="classification_accuracy",
  mode = 'min',
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [9]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="classification_accuracy",
  mode = 'min',
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [10]:
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=1,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

I0000 00:00:1742069545.484695    2243 service.cc:148] XLA service 0x7f49fc008640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742069545.484829    2243 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
I0000 00:00:1742069545.493775    2243 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-15 20:12:25.530709: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742069545.564021    2243 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-03-15 20:12:25.572627: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:794 : INVALID_ARGUMENT: Incompatible shapes: [64,128,128,1] vs. [4,128,128,1]

Stack trace for op definition: 
dummy_file_name:10:dummy_function_name

	 [[{{function_nod

InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/add_4/BroadcastGradientArgs defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/root/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/root/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/root/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/root/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/root/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dispat

In [ ]:
raise Exception("End")

Exception: End

In [ ]:
model.save(f"/workspaces/dev/models/R-CNN/checkpoints/pill_net_sub_{datetime.now().strftime("%Y_%m_%d_sub")}.keras")